In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import folium.plugins

In [3]:
area_centers = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'areas.csv'), index_col=0)
rental = pd.read_csv((Path.cwd().parent / 'data' / 'interim' / 'rental.csv'), parse_dates=['Start_Datetime_Local', 'End_Datetime_Local'])
demand = pd.read_csv((Path.cwd().parent / 'data' / 'interim' / 'openings.csv'), parse_dates=['Created_Datetime_Local'])

In [8]:
date = datetime(2020, 10, 1, 13)
dateend = date+timedelta(seconds=900)

In [9]:
d=demand[(demand['Created_Datetime_Local'] > date) & (demand['Created_Datetime_Local'] < dateend)]
r=rental[(rental['Start_Datetime_Local'] > date) & (rental['Start_Datetime_Local'] < dateend)]
loc = rental[rental['End_Datetime_Local'] <= date]
loc = loc.drop_duplicates(subset='Vehicle_Number_Plate', keep='last') # Keep the last location
current_trips = rental[(rental['Start_Datetime_Local'] < date) & (rental['End_Datetime_Local'] >= date)] # Cars in use
loc = loc[~loc['Vehicle_Number_Plate'].isin(current_trips['Vehicle_Number_Plate'])] # Filter out cars in use

In [10]:
start_map = folium.Map([55.6785706133019, 12.594427257404426], zoom_start=12, tiles='Stamen Toner')
for name, row in loc.iterrows():
    folium.CircleMarker(
        radius=5,
        location=[row['Start_GPS_Latitude'], row['Start_GPS_Longitude']],
        color="blue",
        tooltip=name, 
        fill=True
    ).add_to(start_map)
for name, row in d.iterrows():
    folium.CircleMarker(
        radius=5,
        location=[row['GPS_Latitude'], row['GPS_Longitude']],
        color="crimson",
        tooltip=name,
        fill=False,
    ).add_to(start_map)
for name, row in r.iterrows():
    folium.CircleMarker(
        radius=5,
        location=[row['Start_GPS_Latitude'], row['Start_GPS_Longitude']],
        color="green",
        tooltip=name, 
        fill=True
    ).add_to(start_map)
display(start_map)